In [ ]:
# !pip install dotmap
# !pip install vitaldb 
# !pip install pyPPG==1.0.41
# !pip install openpyxl
# !pip install torch_ecg

In [1]:
import pandas as pd 
import numpy as np
import os 
import sys
import joblib
import matplotlib.pyplot as plt
import torch
# from tqdm import tqdm
from tqdm.notebook import tqdm 
from linearprobing.utils import resample_batch_signal, load_model_without_module_prefix, get_data_for_ml
from preprocessing.ppg import preprocess_one_ppg_signal
from segmentations import waveform_to_segments, save_segments_to_directory
from sklearn.model_selection import train_test_split
from torch_ecg._preprocessors import Normalize
from models.resnet import ResNet1D, ResNet1DMoE
from linearprobing.feature_extraction_papagei import save_embeddings
from linearprobing.extracted_feature_combine import segment_avg_to_dict
from linearprobing.regression import regression_model

/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: module 'numpy' has no attribute 'dtypes'

In [3]:
import numpy as np
print(np.__file__)
print(np.__version__)



/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/numpy/__init__.py
1.23.2


In [3]:
import numpy as np
print(np.__version__)
print(np.__file__)


1.26.4
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/numpy/__init__.py


In [4]:
import torch_ecg
print(torch_ecg.__version__)

0.0.31


## 1. Data

- **(a)**: Download the PPG-BP data from [PPG-BP Database](https://figshare.com/articles/dataset/PPG-BP_Database_zip/5459299) to a directory.
  - Add the download path to `download_dir`.
- **(b)**: Use the download path for further pre-processing.
- **(c)**: Use pre-defined user splits for later linear evaluation


#### 1. (a)

In [4]:
download_dir = "data/5459299"

In [5]:
df = pd.read_excel(f"{download_dir}/Data File/PPG-BP dataset.xlsx", header=1)

In [6]:
subjects = df.subject_ID.values
main_dir = f"{download_dir}/Data File/0_subject/"
ppg_dir = f"{download_dir}/Data File/ppg/"

if not os.path.exists(ppg_dir):
    os.mkdir(ppg_dir)
    
fs = 1000 
fs_target = 125

#### 1. (b)

In [7]:
filenames = [f.split("_")[0] for f in os.listdir(main_dir)]

In [8]:
norm = Normalize(method='z-score')

progress = tqdm(filenames)

for f in progress:
    segments = []
    for s in range(1, 4):
        progress.set_postfix({"processing": f"{f}_{s}"})
        
        signal = pd.read_csv(f"{main_dir}{f}_{str(s)}.txt", sep='\t', header=None)
        signal = signal.values.squeeze()[:-1]
        signal, _ = norm.apply(signal, fs=fs)
        signal, _, _, _ = preprocess_one_ppg_signal(
            waveform=signal,
            frequency=fs
        )
        signal = resample_batch_signal(
            signal, fs_original=fs, fs_target=fs_target, axis=0
        )
        
        padding_needed = 1250 - len(signal)
        pad_left = padding_needed // 2
        pad_right = padding_needed - pad_left
        signal = np.pad(signal, pad_width=(pad_left, pad_right))
        
        segments.append(signal)

    segments = np.vstack(segments)
    child_dir = f.zfill(4)
    save_segments_to_directory(
        save_dir=ppg_dir,
        dir_name=child_dir,
        segments=segments
    )


  0%|          | 0/657 [00:00<?, ?it/s]

[INFO] 0154 already exists
[INFO] 0062 already exists
[INFO] 0227 already exists
[INFO] 0167 already exists
[INFO] 0158 already exists
[INFO] 0148 already exists
[INFO] 0021 already exists
[INFO] 0134 already exists
[INFO] 0052 already exists
[INFO] 0211 already exists
[INFO] 0095 already exists
[INFO] 0032 already exists
[INFO] 0040 already exists
[INFO] 0084 already exists
[INFO] 0142 already exists
[INFO] 0038 already exists
[INFO] 0416 already exists
[INFO] 0253 already exists


/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html

[INFO] 0008 already exists
[INFO] 0180 already exists
[INFO] 0209 already exists
[INFO] 0239 already exists
[INFO] 0173 already exists
[INFO] 0035 already exists
[INFO] 0141 already exists
[INFO] 0114 already exists
[INFO] 0188 already exists
[INFO] 0029 already exists
[INFO] 0030 already exists
[INFO] 0030 already exists
[INFO] 0174 already exists
[INFO] 0051 already exists
[INFO] 0142 already exists
[INFO] 0244 already exists
[INFO] 0139 already exists
[INFO] 0011 already exists
[INFO] 0026 already exists
[INFO] 0207 already exists
[INFO] 0211 already exists
[INFO] 0176 already exists
[INFO] 0185 already exists
[INFO] 0138 already exists
[INFO] 0172 already exists
[INFO] 0227 already exists
[INFO] 0024 already exists
[INFO] 0163 already exists
[INFO] 0085 already exists
[INFO] 0064 already exists
[INFO] 0230 already exists
[INFO] 0230 already exists
[INFO] 0038 already exists
[INFO] 0144 already exists
[INFO] 0217 already exists
[INFO] 0104 already exists
[INFO] 0016 already exists
[

/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html

...

[INFO] 0190 already exists
[INFO] 0417 already exists
[INFO] 0245 already exists
[INFO] 0041 already exists
[INFO] 0219 already exists
[INFO] 0066 already exists
[INFO] 0144 already exists
[INFO] 0061 already exists
[INFO] 0088 already exists
[INFO] 0120 already exists
[INFO] 0128 already exists
[INFO] 0084 already exists
[INFO] 0018 already exists
[INFO] 0065 already exists
[INFO] 0065 already exists
[INFO] 0031 already exists
[INFO] 0163 already exists
[INFO] 0410 already exists


/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html

[INFO] 0224 already exists
[INFO] 0062 already exists
[INFO] 0013 already exists
[INFO] 0155 already exists
[INFO] 0253 already exists
[INFO] 0153 already exists
[INFO] 0087 already exists
[INFO] 0175 already exists
[INFO] 0149 already exists
[INFO] 0166 already exists
[INFO] 0189 already exists
[INFO] 0160 already exists
[INFO] 0155 already exists
[INFO] 0237 already exists
[INFO] 0030 already exists
[INFO] 0256 already exists
[INFO] 0131 already exists
[INFO] 0089 already exists
[INFO] 0023 already exists
[INFO] 0002 already exists
[INFO] 0219 already exists


/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html

[INFO] 0415 already exists
[INFO] 0245 already exists
[INFO] 0063 already exists


#### 1. (c)

In [9]:
df = df.rename(columns={"Sex(M/F)": "sex",
                   "Age(year)": "age",
                   "Systolic Blood Pressure(mmHg)": "sysbp",
                   "Diastolic Blood Pressure(mmHg)": "diasbp",
                   "Heart Rate(b/m)": "hr",
                   "BMI(kg/m^2)": "bmi"})
df = df.fillna(0)

In [10]:
# These randomly selected subject splits used in our work.
# We hardcode them because we cannot share it as a "data source".

train_ids = [  2,   6,   8,  10,  12,  15,  16,  17,  18,  19,  22,  23,  26,
        31,  32,  34,  35,  38,  40,  45,  48,  50,  53,  55,  56,  58,
        60,  61,  63,  65,  66,  83,  85,  87,  89,  92,  93,  97,  98,
        99, 100, 104, 105, 106, 107, 112, 113, 114, 116, 120, 122, 126,
       128, 131, 134, 135, 137, 138, 139, 140, 141, 146, 148, 149, 152,
       153, 154, 158, 160, 162, 164, 165, 167, 169, 170, 175, 176, 179,
       183, 184, 186, 188, 189, 190, 191, 193, 196, 197, 199, 205, 206,
       207, 209, 210, 212, 216, 217, 218, 223, 226, 227, 230, 231, 233,
       234, 240, 242, 243, 244, 246, 247, 248, 256, 257, 404, 407, 409,
       412, 414, 415, 416, 417, 419]

test_ids = [14,  21,  25,  51,  52,  62,  67,  86,  90,  96, 103, 108, 110,
       119, 123, 124, 130, 142, 144, 157, 172, 173, 174, 180, 182, 185,
       192, 195, 200, 201, 211, 214, 219, 221, 228, 239, 250, 403, 405,
       406, 410]

val_ids = [3,  11,  24,  27,  29,  30,  41,  43,  47,  64,  88,  91,  95,
       115, 125, 127, 136, 145, 155, 156, 161, 163, 166, 178, 198, 203,
       208, 213, 215, 222, 229, 232, 235, 237, 241, 245, 252, 254, 259,
       411, 418]

df_train = df[df.subject_ID.isin(train_ids)]
df_val = df[df.subject_ID.isin(val_ids)]
df_test = df[df.subject_ID.isin(test_ids)]

df_train.to_csv(f"{download_dir}/Data File/train.csv", index=False)
df_val.to_csv(f"{download_dir}/Data File/val.csv", index=False)
df_test.to_csv(f"{download_dir}/Data File/test.csv", index=False)

/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/pandas/core/algorithms.py:516: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common = np.find_common_type([values.dtype, comps.dtype], [])


## 2. Extracting Features

In this section, we describe how to load the *PaPaGei* model and extract embeddings.
- **(a)**: Loading a model and extract embeddings from a single signal. This code will be most relevant to customize for your own problems.
- **(b)**: Function to extract features from all ppg segments (batched).
- **(c)**: Extracting *PaPaGei-S* features for PPG-BP.
- **(d)**: Extracting *PaPaGei-S sVRI only* features for PPG-BP.
- **(e)**: Extracting *PaPaGei-P* features for PPG-BP.

In [11]:
batch_size = 256
device = "cuda:0"
case_name = "subject_ID"
ppg_dir = f"{download_dir}/Data File/ppg/"

df_train = pd.read_csv(f"{download_dir}/Data File/train.csv")
df_val = pd.read_csv(f"{download_dir}/Data File/val.csv")
df_test = pd.read_csv(f"{download_dir}/Data File/test.csv")

df_train.loc[:, case_name] = df_train[case_name].apply(lambda x:str(x).zfill(4))
df_val.loc[:, case_name] = df_val[case_name].apply(lambda x:str(x).zfill(4))
df_test.loc[:, case_name] = df_test[case_name].apply(lambda x:str(x).zfill(4))

dict_df = {'train': df_train, 'val': df_val, 'test': df_test}

#### 2. (a) Code to load model and extract embeddings for one signal

**Extend this code to extract features for your own datasets**

In [12]:
model_config = {'base_filters': 32,
            'kernel_size': 3,
            'stride': 2,
            'groups': 1,
            'n_block': 18,
            'n_classes': 512,
            'n_experts': 3
            }

model = ResNet1DMoE(in_channels=1, 
            base_filters=model_config['base_filters'], 
            kernel_size=model_config['kernel_size'],
            stride=model_config['stride'],
            groups=model_config['groups'],
            n_block=model_config['n_block'],
            n_classes=model_config['n_classes'],
            n_experts=model_config['n_experts'])

model_path = "weights/papagei_s.pt"
model = load_model_without_module_prefix(model, model_path)
model.to(device)

ResNet1DMoE(
  (first_block_conv): MyConv1dPadSame(
    (conv): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
  )
  (first_block_bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_block_relu): ReLU()
  (basicblock_list): ModuleList(
    (0): BasicBlock(
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (do1): Dropout(p=0.5, inplace=False)
      (conv1): MyConv1dPadSame(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      )
      (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (do2): Dropout(p=0.5, inplace=False)
      (conv2): MyConv1dPadSame(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      )
      (max_pool): MyMaxPool1dPadSame(
        (max_pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): BasicBlock(
      (bn1): Bat

In [13]:
# load a signal
# note that that signal is already resampled and normalized
signal = joblib.load(os.path.join(ppg_dir, '0002', '0.p'))
signal = torch.Tensor(signal)
signal = signal[None, :].unsqueeze(dim=0)
print(f"PPG dimensions before inference : {signal.shape}")

model.eval()
with torch.inference_mode():
    signal = signal.to(device)
    outputs = model(signal)
    embeddings = outputs[0].cpu().detach().numpy()
print(f"Embedding dimensions : {embeddings.shape}")

PPG dimensions before inference : torch.Size([1, 1, 1250])
Embedding dimensions : (1, 512)


#### 2. (b) Extracting features for all ppg data and saving them for downstream use

In [14]:
def extract_features_and_save(model, ppg_dir, batch_size, device, output_idx, resample, normalize, fs, fs_target, content):
    """
    Function to extract features and save them
    """
    for split in ['train', 'val', 'test']:
        # Choose one split at a time
        df = dict_df[split]
        save_dir = f"{download_dir}/features"

        # Creating require directory structure and names
        if not os.path.exists(f"{save_dir}"):
            os.mkdir(f"{save_dir}")
        
        model_name = model_path.split("/")[-1].split(".pt")[0]
        if not os.path.exists(f"{save_dir}/{model_name}"):
            os.mkdir(f"{save_dir}/{model_name}")
        split_dir = f"{save_dir}/{model_name}/{split}/"
        
        child_dirs = np.unique(df[case_name].values)

        # Function that extracts and saves embeddings
        save_embeddings(path=ppg_dir,
                        child_dirs=child_dirs, 
                        save_dir=split_dir, 
                        model=model, 
                        batch_size=batch_size, 
                        device=device, 
                        output_idx=output_idx,
                        resample=resample, 
                        normalize=normalize, 
                        fs=fs, 
                        fs_target=fs_target)
        
        # Compile the extracted embeddings at the patient or segment level adn save it               
        dict_feat = segment_avg_to_dict(split_dir, content)
        joblib.dump(dict_feat, f"{save_dir}/{model_name}/dict_{split}_{content}.p")

#### 2. (c) Extraction: PaPaGei-S

In [15]:
model_config = {'base_filters': 32,
            'kernel_size': 3,
            'stride': 2,
            'groups': 1,
            'n_block': 18,
            'n_classes': 512,
            'n_experts': 3
            }

model = ResNet1DMoE(in_channels=1, 
            base_filters=model_config['base_filters'], 
            kernel_size=model_config['kernel_size'],
            stride=model_config['stride'],
            groups=model_config['groups'],
            n_block=model_config['n_block'],
            n_classes=model_config['n_classes'],
            n_experts=model_config['n_experts'])

model_path = "weights/papagei_s.pt"
model = load_model_without_module_prefix(model, model_path)
model.to(device)

ResNet1DMoE(
  (first_block_conv): MyConv1dPadSame(
    (conv): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
  )
  (first_block_bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_block_relu): ReLU()
  (basicblock_list): ModuleList(
    (0): BasicBlock(
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (do1): Dropout(p=0.5, inplace=False)
      (conv1): MyConv1dPadSame(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      )
      (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (do2): Dropout(p=0.5, inplace=False)
      (conv2): MyConv1dPadSame(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      )
      (max_pool): MyMaxPool1dPadSame(
        (max_pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): BasicBlock(
      (bn1): Bat

In [16]:
extract_features_and_save(model=model,
                         ppg_dir=ppg_dir,
                         batch_size=batch_size,
                         device=device,
                         output_idx=0,
                         resample=False,
                         normalize=False,
                         fs=125,
                         fs_target=125,
                         content="patient"
                         )

[INFO] data/5459299/features/papagei_s/train/ already exists


 11%|█▏        | 14/123 [00:00<00:00, 133.15it/s]

[INFO] Saving file 0002 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0006 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0008 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0010 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0012 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0015 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0016 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0017 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0018 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0019 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0022 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0023 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0026 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0031 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0032 to data/5459299/features/papagei_s/train/
[INFO] Sav

 26%|██▌       | 32/123 [00:00<00:00, 156.69it/s]

[INFO] Saving file 0066 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0083 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0085 to data/5459299/features/papagei_s/train/
[INFO] Saving file 0087 to data/5459299/features/papagei_s/train/


...

 46%|████▋     | 19/41 [00:00<00:00, 189.36it/s]

[INFO] Saving file 0025 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0051 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0052 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0062 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0067 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0086 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0090 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0096 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0103 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0108 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0110 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0119 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0123 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0124 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0130 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0142 t

100%|██████████| 41/41 [00:00<00:00, 205.66it/s]

[INFO] Saving file 0403 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0405 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0406 to data/5459299/features/papagei_s/test/
[INFO] Saving file 0410 to data/5459299/features/papagei_s/test/



100%|██████████| 41/41 [00:00<00:00, 18251.59it/s]


#### 2. (d) Extraction: PaPaGei-S sVRI only

In [17]:
model_config = {'base_filters': 32,
        'kernel_size': 3,
        'stride': 2,
        'groups': 1,
        'n_block': 18,
        'n_classes': 512,
        }

model = ResNet1D(in_channels=1, 
            base_filters=model_config['base_filters'], 
            kernel_size=model_config['kernel_size'],
            stride=model_config['stride'],
            groups=model_config['groups'],
            n_block=model_config['n_block'],
            n_classes=model_config['n_classes'],
            use_mt_regression=False,
            use_projection=False)

model_path = "weights/papagei_s_svri.pt"
model = load_model_without_module_prefix(model, model_path)
model.to(device)

ResNet1D(
  (first_block_conv): MyConv1dPadSame(
    (conv): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
  )
  (first_block_bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_block_relu): ReLU()
  (basicblock_list): ModuleList(
    (0): BasicBlock(
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (do1): Dropout(p=0.5, inplace=False)
      (conv1): MyConv1dPadSame(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      )
      (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (do2): Dropout(p=0.5, inplace=False)
      (conv2): MyConv1dPadSame(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      )
      (max_pool): MyMaxPool1dPadSame(
        (max_pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): BasicBlock(
      (bn1): BatchN

In [18]:
extract_features_and_save(model=model,
                         ppg_dir=ppg_dir,
                         batch_size=batch_size,
                         device=device,
                         output_idx=0,
                         resample=False,
                         normalize=False,
                         fs=125,
                         fs_target=125,
                         content="patient"
                         )

[INFO] data/5459299/features/papagei_s_svri/train/ already exists


 15%|█▌        | 19/123 [00:00<00:00, 185.50it/s]

[INFO] Saving file 0002 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0006 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0008 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0010 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0012 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0015 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0016 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0017 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0018 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0019 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0022 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0023 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0026 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0031 to data/5459299/features/papagei_s_svri/train/
[INFO]

 33%|███▎      | 41/123 [00:00<00:00, 200.67it/s]

[INFO] Saving file 0099 to data/5459299/features/papagei_s_svri/train/
[INFO] Saving file 0100 to data/5459299/features/papagei_s_svri/train/


...

 66%|██████▌   | 27/41 [00:00<00:00, 266.94it/s]

[INFO] Saving file 0108 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0110 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0119 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0123 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0124 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0130 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0142 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0144 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0157 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0172 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0173 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0174 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0180 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0182 to data/5459299/features/papagei_s_svri/test/
[INFO] Saving file 0

100%|██████████| 41/41 [00:00<00:00, 266.29it/s]


[INFO] Saving file 0410 to data/5459299/features/papagei_s_svri/test/


100%|██████████| 41/41 [00:00<00:00, 16787.04it/s]


#### 2. (e) Extraction: PaPaGei-P

In [19]:
model_config = {'base_filters': 32,
                'kernel_size': 3,
                'stride': 2,
                'groups': 1,
                'n_block': 18,
                'n_classes': 512,
                }

model = ResNet1D(in_channels=1, 
            base_filters=model_config['base_filters'], 
            kernel_size=model_config['kernel_size'],
            stride=model_config['stride'],
            groups=model_config['groups'],
            n_block=model_config['n_block'],
            n_classes=model_config['n_classes'])

model_path = "weights/papagei_p.pt"
model = load_model_without_module_prefix(model, model_path)
model.to(device)

ResNet1D(
  (first_block_conv): MyConv1dPadSame(
    (conv): Conv1d(1, 32, kernel_size=(3,), stride=(1,))
  )
  (first_block_bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (first_block_relu): ReLU()
  (basicblock_list): ModuleList(
    (0): BasicBlock(
      (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (do1): Dropout(p=0.5, inplace=False)
      (conv1): MyConv1dPadSame(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      )
      (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (do2): Dropout(p=0.5, inplace=False)
      (conv2): MyConv1dPadSame(
        (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
      )
      (max_pool): MyMaxPool1dPadSame(
        (max_pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): BasicBlock(
      (bn1): BatchN

In [ ]:
extract_features_and_save(model=model,
                         ppg_dir=ppg_dir,
                         batch_size=batch_size,
                         device=device,
                         output_idx=0,
                         resample=False,
                         normalize=False,
                         fs=125,
                         fs_target=125,
                         content="patient"
                         )

## 3. Linear Evaluation

In this section, we use the extracted embeddings to predict diastolic BP, systolic BP, and Average HR.

In [20]:
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [21]:
save_dir = f"{download_dir}/features/"
df_train = pd.read_csv(f"{download_dir}/Data File/train.csv")
df_val = pd.read_csv(f"{download_dir}/Data File/val.csv")
df_test = pd.read_csv(f"{download_dir}/Data File/test.csv")

df_train.loc[:, case_name] = df_train[case_name].apply(lambda x:str(x).zfill(4))
df_val.loc[:, case_name] = df_val[case_name].apply(lambda x:str(x).zfill(4))
df_test.loc[:, case_name] = df_test[case_name].apply(lambda x:str(x).zfill(4))
                                                                          
case_name = "subject_ID"

In [22]:
def regression(save_dir, model_name, content, df_train, df_val, df_test, case_name, label):
    
    dict_train = joblib.load(f"{save_dir}/{model_name}/dict_train_{content}.p")
    dict_val = joblib.load(f"{save_dir}/{model_name}/dict_val_{content}.p")
    dict_test = joblib.load(f"{save_dir}/{model_name}/dict_test_{content}.p")
    
    X_train, y_train, _ = get_data_for_ml(df=df_train,
                                     dict_embeddings=dict_train,
                                     case_name=case_name,
                                     label=label)

    X_val, y_val, _ = get_data_for_ml(df=df_val,
                                         dict_embeddings=dict_val,
                                         case_name=case_name,
                                         label=label)
    
    X_test, y_test, _ = get_data_for_ml(df=df_test,
                                         dict_embeddings=dict_test,
                                         case_name=case_name,
                                         label=label)
    
    X_test = np.concatenate((X_test, X_val))
    y_test = np.concatenate((y_test, y_val))

    estimator = Ridge()
    param_grid = {
        'alpha': [0.1, 1.0, 10.0, 100.0, 1000.0],  # Regularization strength
        'solver': ['auto', 'cholesky', 'sparse_cg']  # Solver to use in the computational routines
    }
    
    results = regression_model(estimator=estimator,
                param_grid=param_grid,
                X_train=X_train,
                y_train=y_train,
                X_test=X_test,
                y_test=y_test)
    return results

In [23]:
import scipy
print(scipy.__version__)

1.9.1


In [24]:
results_papagei_s = regression(save_dir=save_dir,
                                    model_name='papagei_s',
                                    content="patient",
                                    df_train=df_train,
                                    df_val=df_val,
                                    df_test=df_test,
                                    case_name=case_name,
                                    label="diasbp")

results_papagei_svri = regression(save_dir=save_dir,
                                    model_name='papagei_s_svri',
                                    content="patient",
                                    df_train=df_train,
                                    df_val=df_val,
                                    df_test=df_test,
                                    case_name=case_name,
                                    label="hr")

results_papagei_p = regression(save_dir=save_dir,
                                    model_name='papagei_p',
                                    content="patient",
                                    df_train=df_train,
                                    df_val=df_val,
                                    df_test=df_test,
                                    case_name=case_name,
                                    label="sysbp")

Fitting 4 folds for each of 15 candidates, totalling 60 fits


/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ajhz380/miniconda3/envs/papagei_env/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required f

[CV] END .........................alpha=0.1, solver=cholesky; total time=   0.0s
[CV] END .............................alpha=0.1, solver=auto; total time=   0.0s
[CV] END ........................alpha=0.1, solver=sparse_cg; total time=   0.0s
[CV] END .............................alpha=0.1, solver=auto; total time=   0.0s
[CV] END .........................alpha=0.1, solver=cholesky; total time=   0.0s
[CV] END .........................alpha=0.1, solver=cholesky; total time=   0.0s
[CV] END ............................alpha=10.0, solver=auto; total time=   0.0s
[CV] END ............................alpha=10.0, solver=auto; total time=   0.0s
[CV] END ............................alpha=10.0, solver=auto; total time=   0.0s
[CV] END ............................alpha=10.0, solver=auto; total time=   0.0s
[CV] END ........................alpha=10.0, solver=cholesky; total time=   0.0s
[CV] END ........................alpha=10.0, solver=cholesky; total time=   0.0s
[CV] END ...................

In [25]:
print(f"PaPaGei-S Diastolic BP MAE: {results_papagei_s['mae']}")
print(f"PaPaGei-S sVRI HR MAE: {results_papagei_svri['mae']}")
print(f"PaPaGei-P Systolic BP MAE: {results_papagei_p['mae']}")

PaPaGei-S Diastolic BP MAE: 9.258612562970418
PaPaGei-S sVRI HR MAE: 6.703319642601944
PaPaGei-P Systolic BP MAE: 15.156346576969797
